In [1]:
pip install plaid_python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [3]:
import plaid
from flask import Flask
from flask import render_template
from flask import request
from flask import jsonify
import datetime
from datetime import timedelta
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pandas.core.frame import DataFrame
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
import dash_core_components as dcc        #to get components for interactive user interfaces
import dash_html_components as html  
import dash
import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

# 1. Retrieve data from plaid sandbox

##### The tokens below are removed and if needed, the tokens can be found in tokens.txt file.
##### get client_id and secret from https://dashboard.plaid.com/team/keys 
##### get access_token and item_id from https://dashboard.plaid.com/overview/sandbox

In [4]:
app = Flask(__name__)

In [5]:
client = plaid.Client(client_id='',
                      secret='',
                      environment='sandbox')

In [6]:
access_token = ''
public_token = None
item_id = ''

In [7]:
app.app_context().push()

In [8]:
# Retrieve an Item's accounts
@app.route('/accounts', methods=['GET'])
def get_accounts():
  try:
    accounts_response = client.Accounts.get(access_token)
  except plaid.errors.PlaidError as e:
    return jsonify({'error': {'display_message': e.display_message, 'error_code': e.code, 'error_type': e.type } })
  return jsonify({'error': None, 'accounts': accounts_response})

In [9]:
accounts=get_accounts().json

In [10]:
accounts=accounts['accounts']
accounts=accounts['accounts']
accounts

KeyError: 'accounts'

In [ ]:
rows=[]
for entries in accounts:
    account_id=entries['account_id']
    balances=entries['balances']
    mask=entries['mask']
    name=entries['name']
    official_name=entries['official_name']
    subtype=entries['subtype']
    type=entries['type']
    rows.append((account_id,balances,mask,name,official_name,subtype,type))

In [ ]:
df=pd.DataFrame(rows)
df.columns=['account_id','balances','mask','name','official_name','subtype','type']
df.head()

In [ ]:
response = client.Transactions.get(access_token,
                                   start_date='2019-06-01',
                                   end_date='2021-06-01')
transactions = response['transactions']

# Manipulate the count and offset parameters to paginate
# transactions and retrieve all available data
while len(transactions) < response['total_transactions']:
    response = client.Transactions.get(access_token,
                                       start_date='2019-06-01',
                                       end_date='2021-06-01',
                                       offset=len(transactions)
                                      )
    transactions.extend(response['transactions'])

In [ ]:
transactions

In [ ]:
rows=[]
for entries in transactions:
    account_id=entries['account_id']
    amount=entries['amount']
    category=entries['category']
    category_id=entries['category_id']
    date=entries['date']
    iso_currency_code=entries['iso_currency_code']
    merchant_name=entries['merchant_name']
    payment_channel=entries['payment_channel']
    transaction_id=entries['transaction_id']
    transaction_type=entries['transaction_type']
    rows.append((account_id,amount,category,date,iso_currency_code,merchant_name,payment_channel,transaction_id,transaction_type))

In [ ]:
df=pd.DataFrame(rows)
df.columns=['account_id','amount','category','date','iso_currency_code','merchant_name','payment_channel','transaction_id','transaction_type']


In [ ]:
df['category'] =  df['category'].str.get(0)
df.head()

In [ ]:
df.groupby('account_id').count()

In [ ]:
## choose the account_id with the largest amount

In [ ]:
filter=df['account_id']=='KqGV8JJZZDFx9KwNzMZ3hlGdewnkzxF63VWZB'

In [ ]:
df=df[filter]

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.strftime("%A")
df['month'] = df['date'].apply(lambda x: x.strftime('%Y-%m')) 
df["date"] = df["date"].dt.date
df.head()

In [ ]:
##break payment into home rent payment, bill payent and insurance payment.

In [ ]:
filter1=df['category']=='Payment'
df3=df[filter1]
df3.head()

In [ ]:
x=df3.copy()
x=x.reset_index(drop=True)
x_new=x

In [ ]:
for index in range(len(x)):
    x1=x.loc[[index],:]
    x1.amount=415.7
    x1.category='Bill payment'
    x2=x.loc[[index],:]
    x2.amount=831.4
    x2.category='Insurance payment'
    x_new = pd.concat([x_new,x1,x2]).sort_index()

In [ ]:
x_new=x_new[~x_new['category'].isin(["Payment"])]  
x_new.head()

In [ ]:
df=df.append(x_new,ignore_index=True)


In [ ]:
df.replace('Payment','Home rent payment',inplace=True)
df.replace(2078.5,831.4,inplace=True)

In [ ]:
df = df.sort_values(by = 'date')
df=df.reset_index(drop=True)

In [ ]:
df.drop(['account_id','transaction_id'],axis=1,inplace=True)

In [ ]:
df.head()

# 2. Make interactive graphs

In [ ]:
fig1 = px.bar(df, x="weekday", y="amount", color="category", barmode="group",hover_data=['category', 'date'],title="Expenses by week")       
fig1.show()

In [ ]:
fig2 = px.bar(df, x="month", y="amount", color="category", barmode="group",hover_data=['category', 'date'],title="Expenses by month")       
fig2.show()

In [ ]:
fig3 = px.bar(df, x="month", y="amount", color="category", title="Expenses in last 2 years",hover_data=['category', 'date'])
fig3.show()

In [ ]:
fig4 = px.bar(df, x="date", y="amount", color="category", title="expenses in last 2 years",hover_data=['category', 'date'])
fig4.show()

In [ ]:
df1=df.groupby('category',as_index=False).sum()
df1

In [ ]:
billPayment = (df1[df1.category == 'Bill payment']['amount'].iloc[0])/22
FoodandDrink = (df1[df1.category == 'Food and Drink']['amount'].iloc[0])/22
HomeRentpayment = (df1[df1.category == 'Home rent payment']['amount'].iloc[0])/22
InsurancePayment = (df1[df1.category == 'Insurance payment']['amount'].iloc[0])/22
Recreation = (df1[df1.category == 'Recreation']['amount'].iloc[0])/22
Shops = (df1[df1.category == 'Shops']['amount'].iloc[0])/22
Travel = (df1[df1.category == 'Travel']['amount'].iloc[0])/22


In [ ]:
billPayment

In [ ]:
df1=df1.reset_index()

In [ ]:
fig5 = px.pie(df1, values='amount',names='category',title='Expenses by category')
fig5.show()

# 3. Make predictions using machine learning

In [ ]:
df2=df.groupby('date').sum()
df2

In [ ]:
y=df2['amount'].values.reshape(-1,1)

In [ ]:
x = np.arange(88)
x=x.reshape(-1, 1)

In [ ]:
from sklearn.model_selection import train_test_split as tts
xtrain, xtest, ytrain, ytest = tts(x, y, test_size = 0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(xtrain, ytrain)

In [ ]:
ypred = regressor.predict(xtest)

In [ ]:
import statsmodels.api as sm
xopt = np.hstack([np.ones((x.shape[0], 1)), x])
for i in range(xopt.shape[1]):
    pvalues = sm.OLS(y, xopt.astype(np.float64)).fit().pvalues
    mi = np.argmax(pvalues)
    mp = pvalues[mi]
    if mp > 0.05:
        xopt = np.delete(xopt, [mi], 1)
    else:
        break

In [ ]:
xtrain, xtest, ytrain, ytest = tts(xopt, y, test_size = 0.2, random_state = 0)
regressor = LinearRegression()
regressor.fit(xtrain, ytrain)

ypredopt = regressor.predict(xtest)

In [ ]:
list1=ytest.tolist()
list1 = [int(x) for item in list1 for x in item]
list2=ypred.tolist()
list2 = [int(x) for item in list2 for x in item]
list3=ypredopt.tolist()
list3 = [int(x) for item in list3 for x in item]
x = np.arange(18)
x = x.tolist()

In [ ]:
c={"real points" : list1,
   "predicted points before backward elimination" : list2,
   "predicted points after backward elimination":list3,
   "x":x}
data=DataFrame(c)
data.head()

In [ ]:
fig6 = px.line(data, x='x', y=data.columns[0:3],title='Linear regression',labels={'x':'days in the test set','value':'amount'})
fig6.show()

# 4. Calucations based on income

In [ ]:
pip install numpy_financial

In [ ]:
import numpy_financial as npf

## Variable expenses to be considered apart from regular expenses

In [ ]:
carLoans=200
creditCardMinimums = 120
studentLoans=110
otherMortgages=200
childSupportAndObligations=110
PMI=500
propertyTax=150
homeOwnersAssociationFees=60
homeOwnersInsurance=70
otherLoans=100
maxDebtToIncomeRatio=30

## Required variables to be considered before taking loan 

In [ ]:
grossAnnualIncome = 120000
availableFunds = 10000
emergencyFundPercentage = 20
fixedClosingCosts = 1500
termOfMortgageInYears = 10
annualInterestRate = 6
totalEstimatedClosingCosts = 1000
homePriceDepreciablePercentage = 20
yearsToDepreciate = 40

In [ ]:
def ExpectedMaxMonthlyHousePmt(maximumHousingExpensePercentage):
    return ((grossAnnualIncome*maximumHousingExpensePercentage)/100)/12

def MaxMonthlyPmtonDTI(carLoans, creditCardMinimums, studentLoans, childSupportAndObligations, otherMortgages, otherLoans, maxDebtToIncomeRatio):
    currentMonthlyDebts = carLoans + creditCardMinimums + studentLoans + childSupportAndObligations + otherMortgages + otherLoans
    return ((((grossAnnualIncome * maxDebtToIncomeRatio)/100)/12) - currentMonthlyDebts)

def maxPIPmtBasedOnExpenses(propertyTax, homeOwnersInsurance, PMI, homeOwnersAssociationFees, billPayment, FoodandDrink, HomeRentpayment, InsurancePayment, Recreation, Shops, Travel):
     return maxMonthlyPmt -  (propertyTax + homeOwnersInsurance + PMI + homeOwnersAssociationFees + billPayment + FoodandDrink + HomeRentpayment + InsurancePayment + Recreation + Shops + Travel)

In [ ]:
ExpectedMaxMonthlyHousePmt(22)
maxMonthlyPmtOnDTI = MaxMonthlyPmtonDTI(3000,1000,200,2000,100,300,300)
expectedMaxMonthlyHousePmt = ExpectedMaxMonthlyHousePmt(22)
maxMonthlyPmt = min(expectedMaxMonthlyHousePmt, maxMonthlyPmtOnDTI)
maxMonthlyPmtOnDTI
expectedMaxMonthlyHousePmt
maxMonthlyPmt
_maxPIPmtBasedOnExpenses = maxPIPmtBasedOnExpenses(1500,700,200,600,billPayment,FoodandDrink,HomeRentpayment,InsurancePayment,Recreation,Shops,Travel)
_maxPIPmtBasedOnExpenses
expenses = 12*(carLoans + creditCardMinimums + studentLoans + childSupportAndObligations + otherMortgages +\
               otherLoans + propertyTax + homeOwnersInsurance + PMI + homeOwnersAssociationFees + billPayment +\
               FoodandDrink + HomeRentpayment + InsurancePayment +Recreation + Shops + Travel)
savings = max(grossAnnualIncome - expenses,0)
emergencyFund = emergencyFundPercentage/100*savings
netSavings = savings - emergencyFund
attributes = [emergencyFund,netSavings,expenses]
maxPIPmt=netSavings/12
import numpy_financial as numpy

maxHomePriceBasedOnFunds = max(numpy.pv(((annualInterestRate/100)/12),termOfMortgageInYears*12,-1*maxPIPmt,0)+availableFunds,0)
maxPIPmtBasedOnFunds = max(numpy.pmt(((annualInterestRate/100)/12),termOfMortgageInYears*12,((-1*maxHomePriceBasedOnFunds+availableFunds)),0),0)
loanAmountBasedOnMaxPIPmt = max(numpy.pv(((annualInterestRate/100)/12),termOfMortgageInYears*12,-1*maxPIPmt,0),0)
maxHomePrice = max(loanAmountBasedOnMaxPIPmt - totalEstimatedClosingCosts+availableFunds,0)
annualDepreciation = int(((maxHomePrice*homePriceDepreciablePercentage)/100) / yearsToDepreciate)

In [ ]:
loanAmount = 1000000;
annualInterestRate =6
numberOfYears = 10

In [ ]:

monthlyInterestRate = annualInterestRate / 1200

monthlyPayment = loanAmount * monthlyInterestRate / (1
        - 1 / (1 + monthlyInterestRate) ** (numberOfYears * 12))

totalPayment = monthlyPayment * 12 * numberOfYears



In [ ]:
fig9 = px.pie(attributes, values=attributes,names=["Emergency Fund","Net Savings Avaliable For EMI","Expenses",],title='Maximum Affordable Loan:'+str(int(maxHomePriceBasedOnFunds)))

In [ ]:
fig9

## 1. Maximum loan affordability based on the present expenses pattern

##### A) Maximum affordable loan amount 

In [ ]:
print('Maximum loan amount we can afford is:',int(loanAmountBasedOnMaxPIPmt))

##### C) Maximum available savings for EMI per month

In [ ]:
print('Maximum affordable EMI is:',int(maxPIPmtBasedOnFunds))

##### B) Maximum affordable asset price based on the present spending pattern 

In [ ]:
print('Maximum asset price which we can consider is:', int(maxHomePriceBasedOnFunds))

##### D) Maximum asset price we can consider after considering all expenses associated to the purchase

In [ ]:
print('maximum asset price we can consider after considering all expenses associated to it is:', int(maxHomePrice))

##### E) Annual depreciation for the asset

In [ ]:
print('Annual depreciation for the asset is:', int(annualDepreciation))

## 2. Calculating the amount of expenses to be reduced per month, in the case of shortage of money for the EMI for the targeted loan.

##### A)Monthly EMI to be paid for the required amount of loan:

In [ ]:
print('Monthly EMI is:',int(monthlyPayment))

##### B)Total payment paid for the purchase of asset

In [ ]:
print('Total paypent paid for the asset is:',int(totalPayment))

## 3 If surplus monthly funds are available after making EMI payment, how to manage them?

In [ ]:
if monthlyPayment > maxPIPmtBasedOnFunds:
    print('He needs to reduce his expenses to the extent of', int(monthlyPayment - maxPIPmtBasedOnFunds),'per month')
else:
    print('Good news, No need to limit your current expenses. If your ROI is greater than annual rate of interest, avail bank loan and invest as per your available options to increase your individual wealth, else taking loan is not advisable.')

## Monthly reports based on the expenses:

In [ ]:
FoodLimit = 10
billPaymentlimit = 100
HomeRentpaymentlimit = 100000
InsurancePaymentlimit = 1000
Recreationlimit = 100
Shopslimit = 1000
Travellimit = 100

### Monthly Reports

In [ ]:
if FoodandDrink > FoodLimit:
    print('Food expenses are more than food limit')
else:
    print('Good job, Your food expenses are under limt')
if billPayment > billPaymentlimit:
    print('billPayment expenses are more than billPaymentlimit')
else:
    print('Good job, Your bill payment expenses are under limt')
if HomeRentpayment > HomeRentpaymentlimit:
    print('HomeRentpayment expenses are more than HomeRentpaymentlimit')
else:
    print('Good job, Your home rent expenses are under limt')
if InsurancePayment > InsurancePaymentlimit:
    print('InsurancePayment expenses are more than InsurancePaymentlimit')
else:
    print('Good job, Your insurance payment expenses are under limt')
if Recreation > Recreationlimit:
    print('Recreation expenses are more than Recreationlimit')
else:
    print('Good job, Your recreation expenses are under limt')
if Shops > Shopslimit:
    print('Shops expenses are more than Shopslimit')
else:
    print('Good job, Your shopping expenses are under limt')
if Travel > Travellimit:
    print('Travel expenses are more than Travellimit')
else:
    print('Good job, Your travelling expenses are under limt')

# 5. Build APP

In [ ]:
df=df.set_index('date')

In [ ]:
df=df.reset_index()

In [ ]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


In [ ]:
# Build App

external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "e-CashMate"
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.Div([
        html.H1(" 💸📌e-CashMate📈",style={'text-align':'center',"fontSize": "48px", "color": "#FFFFFF"}),
    
        html.P("Data Source: Plaid Sandbox"
               " Transactions of CHASE BANK"    
               " from 2019-06-01 to 2021-06-01",
                style={'color': '#CFCFCF','margin': '4px auto','max-width': '384px','text-align':'center'}),
            ],
        
        style={'background-color':'black','height':'256px','display': 'flex','flex-direction':'column','justify-content':'center'},
    ),
    
    html.Div([
        html.H2('Sample Dataset',style={'text-align':'center'})
    ]),
    
    
    dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],   
    data=df.to_dict('records'),   
    fixed_rows={'headers': True},  
    page_size=50,   
    style_table={'height': '400px', 'overflowY': 'auto'},  
    style_header={
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
        'maxWidth': 50,
    },
    style_cell={
        'minWidth': 30, 'maxWidth': 60, 'width': 45,
        'textAlign': 'center'  
    }
),
    html.Br(),
    html.Br(),
    html.Div([html.H2('Figures',style={'text-align':'center'})]),
    html.Div([
        dcc.Graph(figure = fig5)
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    html.Div([
        dcc.Graph(figure = fig1)   
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    
    html.Div([
        dcc.Graph(figure = fig2)
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    
    html.Div([
        dcc.Graph(figure = fig3)
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    
    
    html.Div([
        html.H2('Machine Learning',style={'text-align':'center'}),
        dcc.Graph(figure = fig6)
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    
    html.Br(),
    html.Br(),
    html.Div([html.H2('Maximum loan affordability based on the present expenses pattern',style={'text-align':'center'})]),
    html.Div([
        dcc.Graph(figure = fig9)
    ],
    style={'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'},
    ),
    
    html.Br(),
    html.Br(),
    html.Div([html.H2('Monthly reports based on the expenses',style={'text-align':'center'})]),
    
    html.Div([
    html.Div([
        html.Div(children="Food Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='foodlimit',
            placeholder='Please enter the food limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button1',n_clicks=0),
        html.Div(id='food_output',style={'padding': '0','margin': '0','line-height': '38px'})
            ],style={'text-align':'center'}),
    
    html.Br(),
    html.Div([
        html.Div(children="Bill Payment Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='billlimit',
            placeholder='Please enter the bill payment limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button2',n_clicks=0),
        html.Div(id='bill_output',style={'padding': '0', 'margin': '0', 'line-height': '38px'})           
    ],style={'text-align':'center'}),
    
    html.Br(),    
    html.Div([
        html.Div(children="Home Rent Payment Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='homelimit',
            placeholder='Please enter the home rent payment limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button3',n_clicks=0),
        html.Div(id='home_output',style={'padding': '0','margin': '0','line-height': '38px'})           
    ],style={'text-align':'center'}),

    html.Br(),
    html.Div([
        html.Div(children="Insurance Payment Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='insurancelimit',
            placeholder='Please enter the insurance payment limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button4',n_clicks=0),
        html.Div(id='insurance_output',style={'padding': '0', 'margin': '0', 'line-height': '38px'})           
    ],style={'text-align':'center'}),
    
    html.Br(),
    html.Div([
        html.Div(children="Recreation Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='recreationlimit',
            placeholder='Please enter the recreation limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button5',n_clicks=0),
        html.Div(id='recreation_output',style={'padding': '0','margin': '0','line-height': '38px'})        
    ],style={'text-align':'center'}),

    html.Br(),
    html.Div([
        html.Div(children="Shopping Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='shoplimit',
            placeholder='Please enter the shopping limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button6',n_clicks=0),
        html.Div(id='shop_output',style={'padding': '0', 'margin': '0', 'line-height': '38px'})    
    ],style={'text-align':'center'}),
    
    html.Br(),
    html.Div([
        html.Div(children="Travel Limit", style={'margin-bottom': '6px','font-weight': 'bold','color': '#079A82'}),
        dcc.Input(
            id='travellimit',
            placeholder='Please enter the travel limit',
            type="number",
            style={'width':'300px'}
            ),
        html.Button('Submit',id='state-button7',n_clicks=0),
        html.Div(id='travel_output',style={'padding': '0','margin': '0','line-height': '38px'})          
    ],style={'text-align':'center'})
    ],
        style= {'margin-bottom': '24px','box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'
})


],

    style={'margin-right': 'auto', 'margin-left': 'auto','max-width': '1024px', 'padding-right': '10px', 'padding-left': '10px', 
           'margin-top': '32px'}
)

@app.callback(
    Output('food_output', 'children'),
    Input('state-button1', 'n_clicks'),
    State('foodlimit', 'value'),
    prevent_initial_call=True

)
    
def update_output_1(n_clicks,foodlimit):

    if float(FoodandDrink) > float(foodlimit) :
        return'Food expenses are more than food limit'
    else:
        return'Good job, Your food expenses are under limit'
    
@app.callback(
    Output('bill_output', 'children'),
    Input('state-button2', 'n_clicks'),
    State('billlimit', 'value'),
    prevent_initial_call=True

)  
    
def update_output_2(n_clicks,billlimit):

    if float(billPayment) > float(billlimit) :
        return'Bill payment expenses are more than bill payment limit'
    else:
        return'Good job, Your bill payment expenses are under limit'

@app.callback(
    Output('home_output', 'children'),
    Input('state-button3', 'n_clicks'),
    State('homelimit', 'value'),
    prevent_initial_call=True

)
    
def update_output_3(n_clicks,homelimit):

    if float(HomeRentpayment) > float(homelimit) :
        return'Home rent payment expenses are more than home rent payment limit'
    else:
        return'Good job, Your home rent expenses are under limit'
    
@app.callback(
    Output('insurance_output', 'children'),
    Input('state-button4', 'n_clicks'),
    State('insurancelimit', 'value'),
    prevent_initial_call=True

)  
    
def update_output_4(n_clicks,insurancelimit):

    if float(InsurancePayment) > float(insurancelimit) :
        return'Insurance payment expenses are more than insurance payment limit'
    else:
        return'Good job, Your insurance payment expenses are under limit'
    
@app.callback(
    Output('recreation_output', 'children'),
    Input('state-button5', 'n_clicks'),
    State('recreationlimit', 'value'),
    prevent_initial_call=True

)
    
def update_output_5(n_clicks,recreationlimit):

    if float(Recreation) > float(recreationlimit) :
        return'Recreation expenses are more than recreation limit'
    else:
        return'Good job, Your recreation expenses are under limit'
    
@app.callback(
    Output('shop_output', 'children'),
    Input('state-button6', 'n_clicks'),
    State('shoplimit', 'value'),
    prevent_initial_call=True

)  
    
def update_output_6(n_clicks,shoplimit):

    if float(Shops) > float(shoplimit) :
        return'Shops expenses are more than shopping limit'
    else:
        return'Good job, Your shopping expenses are under limit'

@app.callback(
    Output('travel_output', 'children'),
    Input('state-button7', 'n_clicks'),
    State('travellimit', 'value'),
    prevent_initial_call=True

)
    
def update_output_7(n_clicks,travellimit):

    if float(Travel) > float(travellimit) :
        return'Travel expenses are more than travel limit'
    else:
        return'Good job, Your travelling expenses are under limit'
    
# Run app and display result
if __name__ == "__main__":
    app.run_server(debug=True,port=8000)

#CLick on the link below to access the "e-CashMate"